# **Delta Lake Z ORDER**

* Data size - 3.71GB
* Total Rows - 160M+
* Total distinct id1, id2 columns - 100
* Cardinality of id1, id2 (perfect not too low now too high) - 1.6M rows per id

* x3: Delta table that was initially created
* x4: Delta table after it's been optimized via small file compaction
* x5: Delta table Z Ordered by id1
* x6: Delta table Z Ordered by id1 and id2
* x7: Delta table liquid clustering

| Version / Optimization         | Query A (id1) \[sec] | Query B (id2) \[sec] | Query C (id1 & id2) \[sec] | File Distribution                | Implementation Time | Files Skipped |
| ------------------------------ | -------------------- | -------------------- | -------------------------- | -------------------------------- | ------------------- | ------------- |
| **Normal (v3)**                | 12.80                | 6.90                 | 6.17                       | 130 MB × 28, 60 MB × 4 (3.71 GB) | –                   | 0 / 32        |
| **Optimize / Compaction (v4)** | 11.40                | 4.80                 | 5.72                       | 1 GB × 3, 780 MB × 1 (3.71 GB)   | 7m 36.8s            | 0 / 4         |
| **Z-Order on id1 (v5)**        | 7.20                 | 12.70                | 2.17                       | 1.2 GB × 2, 1.3 GB × 1 (3.71 GB) | 11m 51.3s           | 2 / 3         |
| **Z-Order on id1 & id2 (v6)**  | 4.15                 | 6.30                 | 1.28                       | 1.2 GB × 2, 1.3 GB × 1 (3.71 GB) | 15m 54.7s           | 1 / 3         |
| **Liquid Clustering (id2)**    | 13.80                | 7.80                 | 4.60                       | 130 MB × 28, 60 MB × 4 (3.71 GB) | 5m 34.8s            | ?             |


In [ ]:
spark.stop() # only run if you want to stop a running spark session

In [18]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import time

import os
import findspark
import pyspark

# # Set environment variables (local paths)
# os.environ["JAVA_HOME"] = "D:/Programs/Java"
# os.environ["HADOOP_HOME"] = "D:/Programs/hadoop"
# os.environ["SPARK_HOME"] = "D:/Programs/spark/spark-3.5.6-bin-hadoop3"  # Adjust if different

# # Initialize findspark
# findspark.init("D:/Programs/spark/spark-3.5.6-bin-hadoop3")

# Create Spark session
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType

In [19]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
# import deltalake
# import levi

In [20]:
builder = SparkSession \
    .builder \
    .appName("DeltaLake Spark Session") \
    .master("local[4]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

spark

In [ ]:
# delta_path = "D:/HPE-PROJECT/deltalake/delta-lake/normal"
# optimized_delta_path = "D:/HPE-PROJECT/deltalake/delta-lake/optimized"
# zorder_path = "D:/HPE-PROJECT/deltalake/delta-lake/zorder"
# liquid_cluster_path = "D:/HPE-PROJECT/deltalake/delta-lake/liquidcuster"

## **Create Delta Lake**

In [21]:
# initially we do not want liquid clustering to be enabled so we have to set it to false
spark.conf.set("spark.databricks.delta.clusteredTable.enableClusteringTablePreview", "false")

In [ ]:
delta_path = "../data/delta-test"

In [ ]:
csv_path = "../raw_data/N_1e7_K_1e2_single.csv"

In [ ]:
# df = (
#     spark.read.format("parquet")
#     .option("header", True)
#     .load(r"D:\Internship\delta-lake\delta_test2\part-00000-1ec6ab41-0aab-49aa-b339-f04245dfac47-c000.snappy.parquet")
# )

In [24]:
# df.show()
df = spark.read.format("csv").option("header", True).load(csv_path)
df.show()

+-----+-----+------------+---+---+-----+---+---+---------+
|  id1|  id2|         id3|id4|id5|  id6| v1| v2|       v3|
+-----+-----+------------+---+---+-----+---+---+---------+
|id019|id039|id0000056956| 54| 25|79908|  1|  6|57.799172|
|id030|id026|id0000030380| 15| 59|42247|  3| 12| 7.312644|
|id046|id009|id0000071155| 52| 26|62841|  4| 11|48.458201|
|id091|id041|id0000005890| 14| 48|20662|  2|  3|78.210290|
|id015|id051|id0000021869| 13|  6| 3601|  1|  6|70.187575|
|id087|id011|id0000022994|  5| 98|54829|  3| 10|49.395266|
|id090|id079|id0000092893|  2| 15|11476|  0|  7| 2.061937|
|id023|id036|id0000052395| 56| 99|81319|  3| 14|61.664015|
|id006|id052|id0000014135| 77| 87|67494|  1|  3|24.189885|
|id012|id060|id0000063474| 23| 39|42573|  2|  0|98.224646|
|id009|id050|id0000045735| 97| 83|24196|  0|  8|54.763220|
|id094|id028|id0000012094| 37| 62| 9843|  0| 12|10.042739|
|id003|id090|id0000079403| 25| 62|33113|  1| 13|84.784790|
|id084|id026|id0000033763| 31| 50|93707|  2|  0|51.07311

In [ ]:
# To cal. the size of the file formed then-
# df.write.mode("overwrite").parquet("temp_df_size")
# import os
# size_bytes = sum(os.path.getsize(os.path.join("temp_df_size", f)) for f in os.listdir("temp_df_size"))
# size_mb = size_bytes / (1024 * 1024)
# print(f"Estimated DataFrame size: {size_mb:.2f} MB")

In [ ]:
df.write.format("delta").save(delta_path)

# 16 files of 238 mb 
#time 22.5s 

#with local[4]
# 4 files 238 mb time 22

In [ ]:
# we are appending data 3 times so that we have good amount of data
# so 3 versions will be created
for i in range(3):  # Change 5 to 6 for 6 times
    df.write.format("delta").mode("append").save(delta_path)
#37 secs 64 parquet created 
#wiht local 4 
# 52 secs 16 files 952 mb 

In [27]:
delta_table = DeltaTable.forPath(spark, delta_path)

In [28]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      3|2025-08-13 10:11:...|  NULL|    NULL|    WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          2|  Serializable|         true|{numFiles -> 4, n...|        NULL|Apache-Spark/4.0....|
|      2|2025-08-13 10:11:...|  NULL|    NULL|    WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          1|  Serializable|         true|{numFiles -> 4, n...|        NULL|Apache-Spark/4.0....|
|      1|2

## **Compact small files**

In [29]:
delta_table = DeltaTable.forPath(spark, delta_path)

In [ ]:
delta_table.optimize().executeCompaction()

#1min 56 secs 952mb file 
# with local 4 
# 2min 9 sec 975 mb

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [31]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      4|2025-08-13 10:14:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          3|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0....|
|      3|2025-08-13 10:11:...|  NULL|    NULL|    WRITE|{mode -> Append, ...|NULL|    NULL|     NULL|          2|     Serializable|         true|{numFiles -> 4, n...|        NULL|Apache-Spark/4.0.

## **Z Order on id1**

In [32]:
delta_table = DeltaTable.forPath(spark, delta_path)

In [ ]:
delta_table.optimize().executeZOrderBy("id1")
# 3 min 975 mb 1 file 

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [34]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      5|2025-08-13 10:17:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          4|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0....|
|      4|2025-08-13 10:14:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          3|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0.

## **Z Order on id1 and id2**

In [35]:
delta_table = DeltaTable.forPath(spark, delta_path)

In [ ]:
delta_table.optimize().executeZOrderBy("id1", "id2")

# 3min 6 sec 952mb

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [37]:
delta_table.history().show()

+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+---------+--------------------+----+--------+---------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      6|2025-08-13 10:21:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          5|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0....|
|      5|2025-08-13 10:17:...|  NULL|    NULL| OPTIMIZE|{predicate -> [],...|NULL|    NULL|     NULL|          4|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Apache-Spark/4.0.

## **Cluster By - Liquid Clustering**

In [ ]:
delta_path_liquid_clustering = ".../data/liquid-test"

In [ ]:
# Read all parquet files in the folder
df2 = (
    spark.read.format("parquet")
    .option("header", True)  # optional, Parquet stores schema internally
    .load(r"../data/data-1g")  # <-- folder instead of single file
)

In [40]:
df2.createOrReplaceTempView("source_data_view")

In [41]:
spark.conf.set("spark.databricks.delta.clusteredTable.enableClusteringTablePreview", "true")

In [ ]:
spark.sql(f"""
    CREATE TABLE liquid_clustered_table
    USING DELTA
    LOCATION '{delta_path_liquid_clustering}'
    TBLPROPERTIES ('delta.columnMapping.mode' = 'name')
    CLUSTER BY (id1)
    AS SELECT * FROM source_data_view
""")


#48secs 8files of 952mb

DataFrame[]

In [43]:
delta_liquid_table = DeltaTable.forPath(spark, delta_path_liquid_clustering)

In [44]:
delta_liquid_table.history().show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      0|2025-08-13 10:29:...|  NULL|    NULL|CREATE TABLE AS S...|{partitionBy -> [...|NULL|    NULL|     NULL|       NULL|  Serializable|         true|{numFiles -> 8, n...|        NULL|Apache-Spark/4.0....|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+-----------

## **Create views for all five versions of the Delta table**

In [45]:
(
    spark.read.format("delta")
    .option("versionAsOf", "3")
    .load(delta_path)
    .createOrReplaceTempView("x3")
)

In [46]:
(
    spark.read.format("delta")
    .option("versionAsOf", "4")
    .load(delta_path)
    .createOrReplaceTempView("x4")
)

In [47]:
(
    spark.read.format("delta")
    .option("versionAsOf", "5")
    .load(delta_path)
    .createOrReplaceTempView("x5")
)

In [48]:
(
    spark.read.format("delta")
    .option("versionAsOf", "6")
    .load(delta_path)
    .createOrReplaceTempView("x6")
)

In [49]:
# for liquid clustering
(
    spark.read.format("delta")
    .option("versionAsOf", "0")
    .load(delta_path_liquid_clustering)
    .createOrReplaceTempView("x7")
)

In [72]:
import deltalake 
import levi

## **query_a benchmarks(id1)**

In [50]:
%%time

spark.sql(
    "select id1, sum(v1) as v1 from x3 where id1 = 'id016' group by id1"
).collect()

CPU times: total: 15.6 ms
Wall time: 2.83 s


[Row(id1='id016', v1=796700.0)]

In [51]:
%%time

spark.sql(
    "select id1, sum(v1) as v1 from x4 where id1 = 'id016' group by id1"
).collect()

CPU times: total: 31.2 ms
Wall time: 2.02 s


[Row(id1='id016', v1=796700.0)]

In [52]:
%%time

spark.sql(
    "select id1, sum(v1) as v1 from x5 where id1 = 'id016' group by id1"
).collect()

CPU times: total: 0 ns
Wall time: 1.67 s


[Row(id1='id016', v1=796700.0)]

In [53]:
%%time

spark.sql(
    "select id1, sum(v1) as v1 from x6 where id1 = 'id016' group by id1"
).collect()

CPU times: total: 15.6 ms
Wall time: 975 ms


[Row(id1='id016', v1=796700.0)]

In [54]:
%%time

spark.sql(
    "select id1, sum(v1) as v1 from x7 where id1 = 'id016' group by id1"
).collect()

CPU times: total: 31.2 ms
Wall time: 1.01 s


[Row(id1='id016', v1=796700.0)]

In [73]:
dt = deltalake.DeltaTable(delta_path, version=6)
levi.delta_file_sizes(dt)

{'num_files_<1mb': 0,
 'num_files_1mb-500mb': 0,
 'num_files_500mb-1gb': 1,
 'num_files_1gb-2gb': 0,
 'num_files_>2gb': 0}

In [74]:
%%time

levi.skipped_stats(dt, filters=[("id1", "=", "'id016'")])

CPU times: total: 15.6 ms
Wall time: 21.5 ms


{'num_files': 1, 'num_files_skipped': 0, 'num_bytes_skipped': np.int64(0)}

## **query_b benchmarks(id2)**

In [55]:
%%time

spark.sql(
    "select id2, sum(v1) as v1 from x3 where id2 = 'id047' group by id2"
).collect()

CPU times: total: 0 ns
Wall time: 1.11 s


[Row(id2='id047', v1=798640.0)]

In [56]:
%%time

spark.sql(
    "select id2, sum(v1) as v1 from x4 where id2 = 'id047' group by id2"
).collect()

CPU times: total: 31.2 ms
Wall time: 1.09 s


[Row(id2='id047', v1=798640.0)]

In [57]:
%%time

spark.sql(
    "select id2, sum(v1) as v1 from x5 where id2 = 'id047' group by id2"
).collect()

CPU times: total: 0 ns
Wall time: 890 ms


[Row(id2='id047', v1=798640.0)]

In [58]:
%%time

spark.sql(
    "select id2, sum(v1) as v1 from x6 where id2 = 'id047' group by id2"
).collect()

CPU times: total: 0 ns
Wall time: 877 ms


[Row(id2='id047', v1=798640.0)]

In [59]:
%%time

spark.sql(
    "select id2, sum(v1) as v1 from x7 where id2 = 'id047' group by id2"
).collect()

CPU times: total: 15.6 ms
Wall time: 1.12 s


[Row(id2='id047', v1=798640.0)]

## **query_c benchmarks(id1, id2)**

In [60]:
%%time

spark.sql(
    "select id1, id2, sum(v1) from x3 where id1 = 'id016' and id2 = 'id047' group by id1, id2"
).collect()

CPU times: total: 15.6 ms
Wall time: 1.34 s


[Row(id1='id016', id2='id047', sum(v1)=8152.0)]

In [61]:
%%time

spark.sql(
    "select id1, id2, sum(v1) from x4 where id1 = 'id016' and id2 = 'id047' group by id1, id2"
).collect()

CPU times: total: 0 ns
Wall time: 1.39 s


[Row(id1='id016', id2='id047', sum(v1)=8152.0)]

In [62]:
%%time

spark.sql(
    "select id1, id2, sum(v1) from x5 where id1 = 'id016' and id2 = 'id047' group by id1, id2"
).collect()

CPU times: total: 31.2 ms
Wall time: 1.19 s


[Row(id1='id016', id2='id047', sum(v1)=8152.0)]

In [63]:
%%time

spark.sql(
    "select id1, id2, sum(v1) from x6 where id1 = 'id016' and id2 = 'id047' group by id1, id2"
).collect()

CPU times: total: 31.2 ms
Wall time: 1.02 s


[Row(id1='id016', id2='id047', sum(v1)=8152.0)]

In [64]:
%%time

spark.sql(
    "select id1, id2, sum(v1) from x7 where id1 = 'id016' and id2 = 'id047' group by id1, id2"
).collect()

CPU times: total: 0 ns
Wall time: 1.03 s


[Row(id1='id016', id2='id047', sum(v1)=8152.0)]

## **Getting to know about data**

In [65]:
%%time

spark.sql("select count(*) as v1 from x4").collect()

CPU times: total: 0 ns
Wall time: 548 ms


[Row(v1=40000000)]

In [66]:
%%time

spark.sql("select id1, count(v1) as v1 from x4 group by id1").collect()

CPU times: total: 0 ns
Wall time: 4.32 s


[Row(id1='id089', v1=399920),
 Row(id1='id080', v1=399336),
 Row(id1='id087', v1=401884),
 Row(id1='id073', v1=397844),
 Row(id1='id064', v1=401416),
 Row(id1='id043', v1=400864),
 Row(id1='id051', v1=401332),
 Row(id1='id045', v1=400600),
 Row(id1='id074', v1=401584),
 Row(id1='id023', v1=399972),
 Row(id1='id006', v1=400156),
 Row(id1='id013', v1=400032),
 Row(id1='id099', v1=399716),
 Row(id1='id055', v1=398588),
 Row(id1='id052', v1=398132),
 Row(id1='id056', v1=402884),
 Row(id1='id093', v1=399936),
 Row(id1='id075', v1=401844),
 Row(id1='id034', v1=399048),
 Row(id1='id036', v1=400900),
 Row(id1='id032', v1=400992),
 Row(id1='id097', v1=401388),
 Row(id1='id059', v1=399456),
 Row(id1='id065', v1=400800),
 Row(id1='id005', v1=398900),
 Row(id1='id003', v1=398412),
 Row(id1='id037', v1=400176),
 Row(id1='id062', v1=399300),
 Row(id1='id094', v1=401076),
 Row(id1='id002', v1=399324),
 Row(id1='id090', v1=399088),
 Row(id1='id046', v1=398628),
 Row(id1='id030', v1=399516),
 Row(id1='

In [67]:
%%time

spark.sql("select count(distinct(id1)) as v1 from x4").collect()

CPU times: total: 0 ns
Wall time: 2.65 s


[Row(v1=100)]

In [68]:
%%time

spark.sql("select id2, count(v1) as v1 from x4 group by id2").collect()

CPU times: total: 0 ns
Wall time: 3.27 s


[Row(id2='id089', v1=397560),
 Row(id2='id080', v1=399100),
 Row(id2='id073', v1=401420),
 Row(id2='id087', v1=400460),
 Row(id2='id064', v1=398952),
 Row(id2='id043', v1=399020),
 Row(id2='id051', v1=398724),
 Row(id2='id045', v1=400964),
 Row(id2='id074', v1=400044),
 Row(id2='id023', v1=400232),
 Row(id2='id013', v1=400936),
 Row(id2='id006', v1=399532),
 Row(id2='id099', v1=398508),
 Row(id2='id055', v1=399712),
 Row(id2='id052', v1=401056),
 Row(id2='id056', v1=400940),
 Row(id2='id093', v1=400940),
 Row(id2='id034', v1=401076),
 Row(id2='id075', v1=398676),
 Row(id2='id036', v1=403380),
 Row(id2='id032', v1=398956),
 Row(id2='id097', v1=399092),
 Row(id2='id059', v1=397944),
 Row(id2='id065', v1=402292),
 Row(id2='id005', v1=400060),
 Row(id2='id003', v1=401180),
 Row(id2='id037', v1=399956),
 Row(id2='id062', v1=401524),
 Row(id2='id094', v1=397600),
 Row(id2='id002', v1=398060),
 Row(id2='id090', v1=399516),
 Row(id2='id046', v1=402476),
 Row(id2='id030', v1=401288),
 Row(id2='

In [69]:
%%time

spark.sql("select count(distinct(id2)) as v1 from x4").collect()

CPU times: total: 31.2 ms
Wall time: 1.96 s


[Row(v1=100)]